# Livelike Basic Usage Demo: Multiple PUMAs

- ACS 2014 - 2019 5-Year Estimates
- UP Baseline Constraints: employment, industry, occupation, education
- Knox County, TN

## Setup

In [1]:
%load_ext watermark
%watermark

Last updated: 2025-01-31T09:50:17.972502-05:00

Python implementation: CPython
Python version       : 3.12.8
IPython version      : 8.31.0

Compiler    : Clang 18.1.8 
OS          : Darwin
Release     : 24.3.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit



In [2]:
import pathlib

import numpy as np
import pandas as pd
from likeness_vitals.vitals import get_censusapikey
from pymedm import batch, diagnostics

from livelike import multi

%watermark -w
%watermark -iv

/Users/41q/gitlab_repos/likeness/likeness-vitals/likeness_vitals/__init__.py:12: FutureWarning: The ``likeness-vitals`` package is transitioning to a modular API for a cleaner top-level namespace. Classes, functions, and attributes that are currently available in the top-level will only be available within their home modules. Both the current and the updated API will be in place for at least two minor releases following the inclusion of this warning. The release at the time of this warning inclusion was ``v1.3.0``, so the current API will be stable until ``v1.5.0`` at the soonest, and the ``v2.0.0`` release will be the absolute deadline. See [https://code.ornl.gov/likeness/likeness-vitals/-/issues/32] for details on how to update dependant packages.
  warnings.warn(


Watermark: 2.5.0

numpy          : 2.0.2
likeness_vitals: 1.4.3.dev6+g2e663aa
pymedm         : 2.1.1.dev4+g785edf4
pandas         : 2.2.3
livelike       : 1.4.2



### Read Census API Key

In [3]:
key = get_censusapikey(pathlib.Path("..", ""))

### Create baseline PUMA representation

The `puma` class stores a representation of the PUMA based on ACS constraints. It contains the microdata and block group/tract estimates and standard errors required for P-MEDM.

#### Target PUMAs: Knox County

In [4]:
pumas = ["4701602", "4701603", "4701604"]

##### Create PUMA objects for AOI (takes a few minutes)

In [5]:
mpu = multi.make_pumas(pumas, censusapikey=key)

## Spatial Allocation with P-MEDM

### Create P-MEDM object

The batch solver (`pymedm.batch.batch_solve()`) converts PUMAs to P-MEDM problems and solves them multi-threaded.

**Inputs (under the hood):**

- ACS vintage year.
- The PUMS household response serial numbers (`serial`).
- The PUMS household sample weights (`wt`)
- Individual-level constraints (`est_ind`)
- Tract and block group level constraints (`est_trt`, `est_bg`)
- Tract and block group level standard errors (`se_trt`, `se_bg`)

In [6]:
pmds = batch.batch_solve(mpu)

In [7]:
# objective values
for puma, pmd in pmds.items():
    print(f"{puma=}: {pmd.res.state.value}")

puma='4701602': -1.2538384403196665
puma='4701603': -1.5199570129609192
puma='4701604': -1.2547039409778815


### Diagnostics

As a quick diagnostic, we measure the proportion of synthetic constraint estimates fitting the ACS 90% margins of error (MOEs), or the `moe_fit_rate`:

In [8]:
for p in pumas:
    mfr = diagnostics.moe_fit_rate(
        mpu[p].est_ind, mpu[p].est_g2, mpu[p].se_g2, pmds[p].almat
    )
    print(f"PUMA {p}: {str(np.round(mfr['moe_fit_rate'], 4))}")

PUMA 4701602: 0.9996
PUMA 4701603: 0.9997
PUMA 4701604: 0.9999


## Population Synthesis

Use Lovelace and Ballas' (2013) "Truncate, Replicate, Sample" method to create 30 realizations of the residential synthetic population:

In [9]:
hs = multi.synthesize_multi(mpu, pmds)
hs

,sim,geoid,count,replicate
h_id,,,,
2015000004223,0,470930049001,1,4701602
2015000050612,0,470930049001,1,4701602
2015000064675,0,470930049001,1,4701602
2015000067973,0,470930049001,3,4701602
2015000075394,0,470930049001,1,4701602
...,...,...,...,...
2019HU1396194,29,470930060032,9,4701604
2019HU1397336,29,470930060032,1,4701604
2019HU1405831,29,470930060032,1,4701604
